In [3]:
import base64
import hmac
import json
import requests
import time
import datetime
import urllib
import os
import uuid
import re

from hashlib import sha1
from hashlib import md5

In [4]:
base_url = "https://us-east-api.yikyakapi.net/api/"
user_agent = "Dalvik/1.6.0 (Linux; U; Android 4.3; Samsung Galaxy S4 - 4.3 - API 18 - 1080x1920 Build/JLS36G)"
# user_agent = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"

def sign_request(page, params):
    key = "EF64523D2BD1FA21F18F5BC654DFC41B"

    #The salt is just the current time in seconds since epoch
    salt = str(int(time.time()))

    #The message to be signed is essentially the request, with parameters sorted
    msg = "/api/" + page
    sorted_params = list(params.keys())
    sorted_params.sort()
    if len(params) > 0:
        msg += "?"
    for param in sorted_params:
        msg += "%s=%s&" % (param, params[param])
    #Chop off last "&"
    if len(params) > 0:
        msg = msg[:-1]

    #the salt is just appended directly
    msg += salt

    #Calculate the signature
    h = hmac.new(key.encode(), msg.encode(), sha1)
    hash = base64.b64encode(h.digest())

    return hash, salt

def get(page, params):
    url = base_url + page

    hash, salt = sign_request(page, params)
    params['hash'] = hash
    params['salt'] = salt

    headers = {
        "User-Agent": user_agent,
        "Accept-Encoding": "gzip",
        #"Cookie": "lat=" + self.location.latitude + "; long=" + self.location.longitude + "; pending=deleted; expires=Thu,01-Jan-1970 00:00:01 GMT;Max-Age=0",
    }
    
    print("===============", url, params, headers)
    return requests.get(url, params=params, headers=headers)

In [11]:
params = {
    "userID": "00000000-0000-0000-0000-000000000000", #self.id
    "userLat": "37.4274745", #self.location.latitude,
    "userLong": "-122.169719" #self.location.longitude,
}

get("getMessages", params).text

('===============', 'https://us-east-api.yikyakapi.net/api/getMessages', {'hash': 'o8FxvJrfV9IUZPcxvqaRSfb46QM=', 'userLong': '-122.169719', 'userID': '00000000-0000-0000-0000-000000000000', 'salt': '1432517549', 'userLat': '37.4274745'}, {'Accept-Encoding': 'gzip', 'User-Agent': 'Dalvik/1.6.0 (Linux; U; Android 4.3; Samsung Galaxy S4 - 4.3 - API 18 - 1080x1920 Build/JLS36G)'})


u'{"messages":[{"messageID":"R\\/55627ac9dbf37cb1551a739c16a5b","message":"Stanford is fucking expensive. ","latitude":37.42,"longitude":-122.18,"time":"2015-05-24 21:28:41","numberOfLikes":5,"type":"0","comments":1,"posterID":"55627bae7af47","handle":null,"hidePin":"1","liked":0,"gmt":1432517321.9007,"reyaked":0,"score":"5.1432517321","readOnly":0,"deliveryID":102},{"messageID":"R\\/55627aaf2ae24e4341191547581b4","message":"When you\'re depressed and someone says, \\"Just be happy.\\" Oh, thanks, I hadn\'t thought of that, you fuck. ","latitude":37.44,"longitude":-122.17,"time":"2015-05-24 21:28:15","numberOfLikes":5,"type":"0","comments":0,"posterID":"55627bae7afff","handle":null,"hidePin":"1","liked":0,"gmt":1432517295.1755,"reyaked":0,"score":"5.1432517295","readOnly":0,"deliveryID":101},{"messageID":"R\\/55627aac6d701e9605b15f735461b","message":"Stupidest thing you\'ve ever done freshman year? ","latitude":37.44,"longitude":-122.15,"time":"2015-05-24 21:28:12","numberOfLikes":-1,"